In [1]:
import os
%pwd

'd:\\MLOps Udemy Krish Naik\\DS-Project\\research'

In [2]:
os.chdir("../")
%pwd

'd:\\MLOps Udemy Krish Naik\\DS-Project'

In [26]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_column: str

In [27]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml,create_dirs
class ConfigurationManager:
    def __init__(self,config_filepath=CONFIG_FILE_PATH,params_fileppath=PARAMS_FILE_PATH,schema_filepath=SCHEMA_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_fileppath)
        self.schema=read_yaml(schema_filepath)

        create_dirs([self.config.artifacts_root])

    def get_model_trainer_config(self)->ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_dirs([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            model_name=config.model_name,
            alpha=params.alpha,
            l1_ratio=params.l1_ratio,
            target_column=schema.name
        )
        return model_trainer_config


In [30]:
import pandas as pd
import os
from src.datascience import logger
from sklearn.linear_model import ElasticNet
import joblib

class ModelTrainer:
    def __init__(self,config:ModelTrainerConfig):
        self.config = config

    def train(self):
        train_df = pd.read_csv(self.config.train_data_path)
        test_df = pd.read_csv(self.config.test_data_path)

        x_train = train_df.drop([self.config.target_column],axis=1)
        x_test = train_df.drop([self.config.target_column],axis=1)

        y_train = train_df[[self.config.target_column]]
        y_test = test_df[[self.config.target_column]]

        elr = ElasticNet(alpha=self.config.alpha,l1_ratio=self.config.l1_ratio,random_state=42)
        elr.fit(x_train,y_train)

        joblib.dump(elr,os.path.join(self.config.root_dir,self.config.model_name))



In [31]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e

[2025-03-30 01:25:55,865:INFO: common: yaml file: <_io.TextIOWrapper name='config\\config.yaml' mode='r' encoding='UTF-8'> loaded succesfully!]
[2025-03-30 01:25:55,869:INFO: common: yaml file: <_io.TextIOWrapper name='params.yaml' mode='r' encoding='UTF-8'> loaded succesfully!]
[2025-03-30 01:25:55,873:INFO: common: yaml file: <_io.TextIOWrapper name='schema.yaml' mode='r' encoding='UTF-8'> loaded succesfully!]
[2025-03-30 01:25:55,875:INFO: common: created directory at: artifacts]
[2025-03-30 01:25:55,877:INFO: common: created directory at: artifacts/model_trainer]
